# Functions

## Loading Packages

In [1]:
using Roots

include("printmat.jl")   #just a function for prettier matrix printing

println4Ps (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

## A Function with One Input and One Output

In [3]:
function fn1(x)
    y = (x-1.1).^2 - 0.5      # .^2 means that x could be an array 
    return y
end

fn1 (generic function with 1 method)

In [4]:
x = [1;1.5] 

y = fn1(x)                   #calling on the function
println("result from the function fn1(x): ")
printmat(y)

println("the output is one vector since x is a vector")

result from the function fn1(x): 
    -0.490
    -0.340

the output is one vector since x is a vector


In [5]:
x2 = -3:0.01:6                        #plotting the function

plot(x2,fn1(x2),label="fn1(x)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 0 
 
 
 10 
 
 
 20 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,118.379 39.9843,119.636 40.5984,120.89 41.2126,122.141 41.8268,123.389 42.4409,124.633 43.0551,125.875 43.6693,127.114 44.2835,128.349 44.8976,129.582 
 45.5118,130.811 46.126,132.037 46.7402,133.261 47.3543,134.481 47.9685,135.698 48.5827,136.912 49.1969,138.123 49.811,139.331 50.4252,140.535 51.0394,141.737 
 51.6535,142.936 52.2677,144.131 52.8819,145.324 53.4961,146.513 54.1102,147.7 54.7244,148.883 55.3386,150.063 55.9528,151.241 56.5669,152.415 57.1811,153.586 
 57.7953,154.754 58.4094,155.919 59.0236,157.081 59.6378,158.239 60.252,159.395 60.8661,160.548 61.4803,161.697 62.0945,162.844 62.7087,163.987 63.3228,165.127 
 63.937,166.265 64.5512,167.399 65.1654,168.53 65.7795,169.658 66.3937,170.783 67.0079,171.905 67.622,173.024 68.2362,174.14 68.8504,175.252 69.4646,176.362 
 70.0787,177.469 70.6929,178.572 71.3071,179.673 71.9213,180.77 72.5354,181.864 73.1496,182.956 73.7638,184.044 74.378,185.129 74.9921,186.211 75.6063,187.29 
 76.2205,188.366 76.8346,189.439 77.4488,190.508 78.063,191.575 78.6772,192.639 79.2913,193.699 79.9055,194.757 80.5197,195.811 81.1339,196.862 81.748,197.911 
 82.3622,198.956 82.9764,199.998 83.5906,201.037 84.2047,202.073 84.8189,203.106 85.4331,204.136 86.0472,205.163 86.6614,206.186 87.2756,207.207 87.8898,208.224 
 88.5039,209.239 89.1181,210.25 89.7323,211.259 90.3465,212.264 90.9606,213.266 91.5748,214.265 92.189,215.261 92.8031,216.254 93.4173,217.244 94.0315,218.231 
 94.6457,219.215 95.2598,220.196 95.874,221.173 96.4882,222.148 97.1024,223.12 97.7165,224.088 98.3307,225.053 98.9449,226.016 99.5591,226.975 100.173,227.931 
 100.787,228.884 101.402,229.834 102.016,230.781 102.63,231.725 103.244,232.666 103.858,233.604 104.472,234.538 105.087,235.47 105.701,236.399 106.315,237.324 
 106.929,238.247 107.543,239.166 108.157,240.082 108.772,240.995 109.386,241.905 110,242.813 110.614,243.717 111.228,244.617 111.843,245.515 112.457,246.41 
 113.071,247.302 113.685,248.19 114.299,249.076 114.913,249.959 115.528,250.838 116.142,251.714 116.756,252.588 117.37,253.458 117.984,254.325 118.598,255.189 
 119.213,256.05 119.827,256.908 120.441,257.763 121.055,258.615 121.669,259.464 122.283,260.309 122.898,261.152 123.512,261.991 124.126,262.828 124.74,263.661 
 125.354,264.492 125.969,265.319 126.583,266.143 127.197,266.964 127.811,267.782 128.425,268.597 129.039,269.409 129.654,270.218 130.268,271.024 130.882,271.826 
 131.496,272.626 132.11,273.422 132.724,274.216 133.339,275.006 133.953,275.794 134.567,276.578 135.181,277.359 135.795,278.137 136.409,278.912 137.024,279.684 
 137.638,280.453 138.252,281.219 138.866,281.982 139.48,282.742 140.094,283.498 140.709,284.252 141.323,285.002 141.937,285.75 142.551,286.494 143.165,287.236 
 143.78,287.974 144.394,288.709 145.008,289.441 145.622,290.17 146.236,290.896 146.85,291.619 147.465,292.339 148.079,293.056 148.693,293.769 149.307,294.48 
 149.921,295.187 150.535,295.892 151.15,296.593 151.764,297.292 152.378,297.987 152.992,298.679 153.606,299.368 154.22,300.054 154.835,300.737 155.449,301.417 
 156.063,302.094 156.677,302.768 157.291,303.438 157.906,304.106 158.52,304.771 159.134,305.432 159.748,306.091 160.362,306.746 160.976,307.398 161.591,308.047 
 162.205,308.694 162.819,309.337 163.433,309.977 164.047,310.614 164.661,311.247 165.276,311.878 165.89,312.506 166.504,313.131 167.118,313.752 167.732,314.371 
 168.346,314.986 168.961,315.599 169.575,316.208 170.189,316.814 170.803,317.417 171.417,318.017 172.031,318.614 172.646,319.208 173.26,319.799 173.874,320.387 
 174.488,320.972 175.102,321.554 175.717,322.132 176.331,322.708 176.945,323.28 177.559,323.85 178.173,324.416 178.787,324.979 179.402,325.539 180.016,326.097 
 180.63,326.651 181.244,327.

## An "Anonymous" Function

The expression 
```
x->(x-1.1).^2 - 0.5
```
creates an anonymous function (a function without a name). It is useful as an input to another function (see below).

In [6]:
x1 = fzero(x->(x-1.1).^2 - 0.5,[-1;1])      #finding a root of a function      
                                            #[-1;1] searches roots in this interval
printlnPs("A root at: ",x1)

A root at: 

## A "one-liner"

We can also create a short version of a function as in the cell below. Notice, however, that it can (essentially) only contain one expression.

In [7]:
fn1b(x) = (x-1.1).^2 - 0.5          #short form of a function

fn1b (generic function with 1 method)

In [8]:
y = fn1b(x)                 
println("result from the function fn1b(x): ")
printmat(y)

result from the function fn1b(x): 
    -0.490
    -0.340



## Elementwise Evaluation

The functions above were written to accept an array as the input (by using .^2) and give an array as output. 

Suppose instead that you have a traditional function that only works on a scalar input (by using ^2 instead of .^2). You can still supply an array as input, provided you use the dot syntax: fn1c.(x)

In [9]:
function fn1c(x)                     #similar to fn1(), but would not work when x is an array
    y = (x-1.1)^2 - 0.5
    return y
end

fn1c (generic function with 1 method)

In [10]:
println("fn1c.(x) evaluates fn1c(x[i,j]) for each element x[i,j]")

y5 = fn1c.(x)

printmat(y5)

fn1c.(x) evaluates fn1c(x[i,j]) for each element x[i,j]
    -0.490
    -0.340



## Several Inputs

It is straightforward to define functions with several inputs. You can also specify default values on the inputs. They are used when you do not supply that input when calling on the function.

In [11]:
function fn2(x,b=1)                 #b=1 is the default in case we call as fn2(x)
    y = b*(x-1.1).^2 - 0.5
    return y
end

fn2 (generic function with 2 methods)

In [12]:
y2 = fn2(x)
println("result from the function fn2(x): ")
printmat(y2)

y2b = fn2(x,7)
println("result from the function fn2(x,7): ")
printmat(y2b)

result from the function fn2(x): 
    -0.490
    -0.340

result from the function fn2(x,7): 
    -0.430
     0.620



## Several Outputs

Functions can preduce a "tuple" (y1,y2,y3) as outputs. (There could be more values)

In case you only want the first two outputs, call as (y1,y2,) = fn().

Instead, if you only want the 2nd and third outputs, call as (_,y2,y3) = fn()

You can also extract the second output as y2 = fn()[2]

In [13]:
function fn3(x,b=1)                 
    y1 = b*(x-1.1).^2 - 0.5
    y2 = b*x
    y3 = 3
    return y1, y2, y3
end

fn3 (generic function with 2 methods)

In [14]:
(y1,y2,) = fn3(x,7)

println("result from the function fn3(x,7): ")
printmat([y1 y2])

println("the output is two vectors since x is a vector")

result from the function fn3(x,7): 
    -0.430     7.000
     0.620    10.500

the output is two vectors since x is a vector


In [15]:
y1, = fn3(x,7)           #to grab just the first output  

y2 = fn3(x,7)[2]         #to grab just the second output from the function

println("the first result from the function fn3(x,7): ")
printmat(y1)
println("result from the function fn3(x,7)[2]: ")
printmat(y2)

the first result from the function fn3(x,7): 
    -0.430
     0.620

result from the function fn3(x,7)[2]: 
     7.000
    10.500



## Documenting Your Function 

To use Julia's help function (```? function name```), use this structure

"""

here is the documentation (in markdown)

"""

function code

The cell below illustrates a simple case.

In [16]:
"""
    fn4(x,b=1)

Calculate b*(x-1.1)^2 - 0.5.

# Arguments
- `x`:    a number
- `b`:    a number

"""
function fn4(x,b=1)       
    y = b*(x-1.1)^2 - 0.5
    return y
end

fn4

In [17]:
? fn4

search:



```
fn4(x,b=1)
```

Calculate b*(x-1.1)^2 - 0.5.

# Arguments

  * `x`:    a number
  * `b`:    a number
